Esta é uma script em jupyter para fazer pequenas funções e depois passar para a script em python.

In [83]:
import os
import subprocess
from Bio import SeqIO, SeqFeature, Entrez, AlignIO, ExPASy, SwissProt
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast import NCBIXML, NCBIWWW
from Bio.Align.Applications import ClustalwCommandline
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor, DistanceCalculator
from Bio import Phylo

### Análise da sequência e das features presentes no NCBI

Funtion to get gene genbank file

In [29]:
def get_seq(gene_id, filename, email="...@gmail.com", db="nucleotide",rettype="gb", output_dir="output"):
    """
    Fetches a sequence from NCBI and saves it to a file in the specified output directory.
    
    Args:
        gene_id (str): NCBI Gene ID.
        email (str): User's email for NCBI access.
        filename (str): File name to save the sequence.
        output_dir (str): Directory where the file will be saved.
    """
    Entrez.email = email
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    output_path = os.path.join(output_dir, filename)  # Full path to the output file

    try:
        print(f"Fetching data for gene ID: {gene_id}...")
        handle = Entrez.efetch(db=db, id=gene_id, rettype=rettype, retmode="text")
        record = SeqIO.read(handle, "genbank")
        handle.close()
        print("Saving the sequence to file...")
        with open(output_path, "w") as output_file:  # Use output_path here
            SeqIO.write(record, output_file, "genbank")
        print(f"Sequence saved successfully to {output_path}.")
        
    except Exception as e:
        print(f"An error occurred: {e}")

In [35]:
get_seq("U10926.1", filename = "comS.gb", output_dir="genes")


Fetching data for gene ID: U10926.1...
An error occurred: No records found in handle


Funtion to extract cds sequence

In [48]:
def extract_cds_from_genbank(genbank_file, filename="cds_sequences.fasta", output_dir="genes_cds"):
    """
    Extracts CDS sequences from a GenBank file and saves them as FASTA.
    
    Args:
        genbank_file (str): Path to the GenBank file.
        output_file (str): Path to save the extracted CDS sequences in FASTA format.
    
    Returns:
        None
    """
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    output_path = os.path.join(output_dir, filename)  # Full path to the output file
    
    cds_count = 0
    with open(output_path, "w") as fasta_output:
        for record in SeqIO.parse(genbank_file, "genbank"):
            for feature in record.features:
                if feature.type == "CDS":
                    # Extract the CDS sequence
                    cds_seq = feature.extract(record.seq)
                    
                    # Get the gene or protein ID from qualifiers
                    gene_id = feature.qualifiers.get("gene", ["Unknown_gene"])[0]
                    protein_id = feature.qualifiers.get("protein_id", ["Unknown_protein"])[0]
                    
                    # Write the CDS to the FASTA file
                    fasta_output.write(f">{gene_id}|{protein_id}\n{cds_seq}\n")
                    cds_count += 1
    
    print(f"{cds_count} CDS sequences extracted and saved to {filename}.")

In [52]:
extract_cds_from_genbank("genes/comS.gb", "comS_cds.fasta")

1 CDS sequences extracted and saved to comS_cds.fasta.


Function to get the annotation from genbank file

In [50]:
def extract_annotations(filename):
    """
    Extracts annotations from a GenBank file.
    
    Args:
        filename (str): Path to the GenBank file.
    """
    try:
        record = SeqIO.read(filename, "genbank")
        print(f"Gene Description: {record.description}")
        print(f"Organism: {record.annotations.get('organism', 'Unknown')}")
        print(f"Annotations: {record.annotations}")

    except Exception as e:
        print(f"An error occurred: {e}")


In [51]:
extract_annotations("genes/comS.gb")

Gene Description: Bacillus subtilis 168 genetic competence regulation (comS) gene, complete cds
Organism: Bacillus subtilis subsp. subtilis str. 168
Annotations: {'molecule_type': 'DNA', 'topology': 'linear', 'data_file_division': 'BCT', 'date': '26-JAN-1995', 'accessions': ['U10926'], 'sequence_version': 1, 'keywords': [''], 'source': 'Bacillus subtilis subsp. subtilis str. 168', 'organism': 'Bacillus subtilis subsp. subtilis str. 168', 'taxonomy': ['Bacteria', 'Bacillati', 'Bacillota', 'Bacilli', 'Bacillales', 'Bacillaceae', 'Bacillus'], 'references': [Reference(title='Identification of comS, a gene of the srfA operon that regulates the establishment of genetic competence in Bacillus subtilis', ...), Reference(title="Nucleotide sequence of 5' portion of srfA that contains the region required for competence establishment in Bacillus subtilus", ...), Reference(title='Direct Submission', ...)]}


Funtion to analyse features and qualifiers

In [53]:
def analyze_features(filename):
    """
    Analyzes features and qualifiers in a GenBank file.
    
    Args:
        filename (str): Path to the GenBank file.
    """
    try:
        record = SeqIO.read(f"genes/{filename}", "genbank")
        print(f"Number of Features: {len(record.features)}")
        for feature in record.features:
            print(f"Type: {feature.type}")
            print(f"Location: {feature.location}")
            print(f"Qualifiers: {feature.qualifiers}")
            print("-" * 50)
            
    except Exception as e:
        print(f"An error occurred: {e}")


In [54]:
analyze_features("comS.gb")

Number of Features: 4
Type: source
Location: [0:570](+)
Qualifiers: {'organism': ['Bacillus subtilis subsp. subtilis str. 168'], 'mol_type': ['genomic DNA'], 'strain': ['168'], 'sub_species': ['subtilis'], 'type_material': ['type strain of Bacillus subtilis'], 'db_xref': ['taxon:224308'], 'map': ['30 degrees'], 'note': ['sequence similar to srfA2 gene, GenBank Accession Number X70356, and srfAB gene, GenBank Accession Number D13262']}
--------------------------------------------------
Type: regulatory
Location: [188:189](+)
Qualifiers: {'regulatory_class': ['ribosome_binding_site']}
--------------------------------------------------
Type: gene
Location: [210:351](+)
Qualifiers: {'gene': ['comS']}
--------------------------------------------------
Type: CDS
Location: [210:351](+)
Qualifiers: {'gene': ['comS'], 'function': ['regulation of genetic competence'], 'experiment': ['experimental evidence, no additional details recorded'], 'codon_start': ['1'], 'transl_table': ['11'], 'organism'

Funtion to extract external references

In [55]:
def extract_external_references(filename):
    """
    Extracts external database references from a GenBank file.
    
    Args:
        filename (str): Path to the GenBank file.
    """
    try:
        record = SeqIO.read(filename, "genbank")
        external_refs = []

        for feature in record.features:
            if "db_xref" in feature.qualifiers:
                external_refs.extend(feature.qualifiers["db_xref"])
        print("External References:")

        for ref in set(external_refs):
            print(ref)
            
    except Exception as e:
        print(f"An error occurred: {e}")



In [56]:
extract_external_references("genes/comS.gb")

External References:
taxon:224308


### Análise de homologias por BLAST

Função para fazer um NCBI blast apartir de um file (pode-se escolher o tipo de blast, database e outros parâmetros)

In [63]:
def blast(file_name, file_format = "fasta", program = "blastn", database = "nt", e_value = 0.05, hitlist_size = 100):
    """
    Realiza o BLAST de uma sequência contida em um arquivo.
    
    Parameters:
        file_name (str): Nome do arquivo contendo a sequência.
        file_format (str): Formato do arquivo (padrão: 'fasta').
        program (str): Programa BLAST a ser usado (padrão: 'blastn').
        database (str): Banco de dados para busca (padrão: 'nt').
        e_value (float): Limite de valor E para busca (padrão: 0.05).
        hitlist_size (int): Número de hits a serem retornados (padrão: 50).
    
    Returns:
        Um handle com os resultados do BLAST, ou None em caso de erro.
    """
    
    try:
        #lê o arquivo
        record = SeqIO.read(open(file_name), format = file_format)

        #Começa o blast remoto
        print("BLASTing...")
        result_handle = NCBIWWW.qblast(program, database, record.format("fasta"), expect = e_value, hitlist_size = hitlist_size)

        #returns result handle
        print("BLAST concluído com sucesso.")
        return result_handle
    
    except Exception as e:
        print(f"Erro ao executar o BLAST: {e}")
        return None

Função que utiliza função blast, mas guardar o output num file.

In [62]:
def get_blast(file_name, output_name = "blast_result", output_dir="blast_output", file_format = "fasta", program = "blastn", database = "nt", e_value = 0.01, hitlist_size = 100):
    """
    Realiza o BLAST usando a função `blast` e salva os resultados em um arquivo.
    
    Parameters:
        file_name (str): Nome do arquivo contendo a sequência.
        output_name (str): Nome do arquivo para salvar os resultados.
        Outros parâmetros são os mesmos da função `blast`.
    """

    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    output_path = os.path.join(output_dir, output_name)  # Full path to the output file

    result_handle = blast(file_name, file_format, program, database, e_value, hitlist_size)

    if result_handle is not None:
        try:
            # Salva os resultados em um arquivo XML
            with open(f"{output_path}.xml", "w") as save_file:
                save_file.write(result_handle.read())
                print(f"Resultados salvos em {output_name}.xml.")

        except Exception as e:
            print(f"Erro ao salvar os resultados: {e}")

        finally:
            result_handle.close()
    

Teste da função *get_blast* 

In [64]:
get_blast("genes_cds/comS_cds.fasta", output_name ="comS_cds", hitlist_size=20000)

BLASTing...
BLAST concluído com sucesso.
Resultados salvos em comS_cds.xml.


Function to parse a blast result

In [65]:
def parse_blast_results(file_name, exclude=None, e_value_thresh=0.05, identity_thresh=90):
    """
    Parses BLAST results and extracts significant hits, excluding specific species or TaxIDs.
    
    Args:
        file_name (str): Path to the BLAST result file in XML format.
        exclude_taxid (int or None): TaxID of the species to exclude.
        exclude_species (str or None): Species name to exclude (e.g., "Bacillus subtilis").
        e_value_thresh (float): Threshold for E-value significance.
        identity_thresh (float): Minimum percentage identity for significant hits.
    
    Returns:
        list: A list of dictionaries containing significant hit information.
    """
    from Bio.Blast import NCBIXML

    with open(file_name) as result_handle:
        blast_record = NCBIXML.read(result_handle)
    
    significant_hits = []
    for alignment in blast_record.alignments:
        EXC = False
        if exclude:
            for _ in exclude:
                if _.lower() in alignment.title.lower():
                    EXC = True
                    break  # Exit the loop once a match is found
        
        if not EXC:
            for hsp in alignment.hsps:
                if hsp.expect < e_value_thresh and (hsp.identities / hsp.align_length) * 100 > identity_thresh:
                    hit_info = {
                        "title": alignment.title,
                        "length": alignment.length,
                        "e_value": hsp.expect,
                        "identity": (hsp.identities / hsp.align_length) * 100,
                        "alignment_length": hsp.align_length
                    }
                    significant_hits.append(hit_info)
                    
    
    return significant_hits

In [67]:
parse_blast_results("blast_output/comS_cds.xml", exclude=["subtilis","chromosome","complete", "genome"], e_value_thresh=0.05, identity_thresh=50)

[{'title': 'gi|1257575185|dbj|LC171348.1| Bacillus sp. FW1 genes for pyrene metabolism, contig_7',
  'length': 455392,
  'e_value': 5.1292e-61,
  'identity': 98.58156028368793,
  'alignment_length': 141},
 {'title': 'gi|1757450143|gb|MK570508.1| Bacillus amyloliquefaciens strain TSBSO3.8 surfactin gene region',
  'length': 65411,
  'e_value': 7.136e-34,
  'identity': 84.39716312056737,
  'alignment_length': 141},
 {'title': 'gi|1757450197|gb|MK570509.1| Bacillus amyloliquefaciens strain H2O-1 surfactin gene region',
  'length': 65415,
  'e_value': 7.136e-34,
  'identity': 84.39716312056737,
  'alignment_length': 141},
 {'title': 'gi|2784321086|gb|CP165606.1| Bacillus velezensis strain BP1 plasmid unnamed',
  'length': 4056860,
  'e_value': 3.0343e-32,
  'identity': 83.68794326241135,
  'alignment_length': 141},
 {'title': 'gi|42820782|emb|AJ575642.1| Bacillus amyloliquefaciens yciC gene, yx01 gene, yckc gene, yckD gene, yckE gene, nin gene, nuc gene, hxlB gene, hxlA gene, hxlR gene, xy

### Alinhamento múltiplo e filogenia

Função de extract de sequências

In [69]:
def extract_sequences_from_blast(file_name, query_fasta, exclude=None, e_value_thresh=0.05, identity_thresh=90, n_seq=20):
    """
    Extracts sequences from BLAST results for alignment, using a query FASTA file.
    
    Args:
        file_name (str): Path to the BLAST XML result file.
        query_fasta (str): Path to the query sequence FASTA file.
        exclude (list): List of species or keywords to exclude.
        e_value_thresh (float): Threshold for E-value significance.
        identity_thresh (float): Minimum percentage identity for significant hits.
        n_seq (int): Maximum number of sequences to extract.
    
    Returns:
        dict: Dictionary with sequence titles as keys and sequences as values, including the query sequence.
    """
    # Read the query sequence from the FASTA file
    query_sequence = None
    with open(query_fasta, "r") as fasta_handle:
        query_record = SeqIO.read(fasta_handle, "fasta")
        query_sequence = str(query_record.seq)
    
    if not query_sequence:
        raise ValueError("Query sequence could not be read from the FASTA file.")
    
    # Initialize the dictionary with the query sequence
    sequences = {"Query": query_sequence}

    # Parse the BLAST XML file
    with open(file_name) as result_handle:
        blast_record = NCBIXML.read(result_handle)

    for alignment in blast_record.alignments:
        # Check exclusion criteria
        if exclude and any(term.lower() in alignment.title.lower() for term in exclude):
            continue  # Skip this alignment if it matches exclusion criteria
        
        for hsp in alignment.hsps:
            # Check E-value and identity thresholds
            if hsp.expect < e_value_thresh and (hsp.identities / hsp.align_length) * 100 > identity_thresh:
                sequences[alignment.title] = hsp.sbjct  # Add the sequence
                if len(sequences) >= n_seq:  # Stop if desired number of sequences is reached
                    return sequences
    
    return sequences

In [72]:
sequences=extract_sequences_from_blast("blast_output/comS_cds.xml", query_fasta="genes_cds/comS_cds.fasta",
                                        exclude=["subtilis","chromosome","complete", "genome"], e_value_thresh=0.05, identity_thresh=50)

In [101]:
def save_to_fasta(sequences, output_file="sequences.fasta",output_dir="homologs"):
    """
    Saves sequences to a FASTA file.
    
    Args:
        sequences (dict): Dictionary of sequences with titles as keys.
        output_file (str): Name of the FASTA file.
    """
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    output_path = os.path.join(output_dir, output_file)  # Full path to the output file

    records = [SeqRecord(Seq(seq), id=title, description="") for title, seq in sequences.items()]
    SeqIO.write(records, output_path, "fasta")
    print(f"Sequences saved to {output_path}.")



In [102]:
# Example Usage
save_to_fasta(sequences, "comS_homologs.fasta")

Sequences saved to homologs\comS_homologs.fasta.


In [85]:
def run_clustalO(input_file, output_file="aligned_sequences.aln", output_dir="clustal_output"):
    """
    Runs Clustal Omega for multiple sequence alignment.
    
    Args:
        input_file (str): Path to the input FASTA file.
        output_file (str): Name of the output alignment file.
        output_dir (str): Directory to save the alignment result.
    
    Returns:
        str: Full path to the output alignment file.
    """
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist
    output_path = os.path.join(output_dir, output_file)  # Full path to the output file

    clustalo_exe = "clustal-omega-1.2.2-win64\\clustalo"  # Path to Clustal Omega executable

    # Command to run Clustal Omega
    command = [
        clustalo_exe,
        "--infile", input_file,
        "--outfile", output_path,
        "--outfmt", "clu",  # Output format: Clustal
        "--force"  # Overwrite existing files if necessary
    ]

    try:
        # Run the command
        result = subprocess.run(command, capture_output=True, text=True, check=True)
        print("Clustal Omega alignment completed successfully.")
        print(result.stdout)
        return output_path
    except subprocess.CalledProcessError as e:
        print("Error during Clustal Omega execution:")
        print(e.stderr)
        raise



In [87]:
aligned_file = run_clustalO(input_file="comS_homologs.fasta", output_file="comS.aln", output_dir="clustal_output")


Clustal Omega alignment completed successfully.



Função para costrução de árvores apartir do output do clustalOmega

In [94]:
def build_trees(file_name, file_type, matrix="blosum62", output_dir="tree_output"):
    """
    Builds phylogenetic trees (UPGMA and NJ) from a sequence alignment file.
    
    Args:
        file_name (str): Path to the input alignment file.
        file_type (str): Format of the input alignment file (e.g., "clustal", "fasta").
        matrix (str): Scoring matrix for distance calculation (default: "blosum62").
        output_dir (str): Directory to save intermediate files and outputs (default: "tree_output").
    
    Returns:
        None: Displays the trees as ASCII diagrams in the console.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    try:
        # Read the alignment file
        alignment = AlignIO.read(file_name, file_type)
        
        # Save the alignment in Stockholm format for reference
        stockholm_file = os.path.join(output_dir, "alignment.sth")
        AlignIO.write(alignment, stockholm_file, "stockholm")
        print(f"Alignment saved in Stockholm format at: {stockholm_file}")

        # Calculate pairwise distances
        calculator = DistanceCalculator(matrix)
        dm = calculator.get_distance(alignment)
        
        # Construct trees using UPGMA and NJ methods
        constructor = DistanceTreeConstructor()
        upgma_tree = constructor.upgma(dm)
        nj_tree = constructor.nj(dm)

        # Display trees as ASCII
        print("Neighbor-Joining Tree:")
        Phylo.draw_ascii(nj_tree, column_width=120)
        print("-" * 50)
        print("UPGMA Tree:")
        Phylo.draw_ascii(upgma_tree, column_width=120)

    except Exception as e:
        print(f"An error occurred: {e}")

In [96]:
build_trees(
    file_name="clustal_output/comS.aln",
    file_type="clustal",
    matrix="blosum62",
    output_dir="tree_output"
)

Alignment saved in Stockholm format at: tree_output\alignment.sth
Neighbor-Joining Tree:
                         _______________________________________________________________ gi|3046719|emb|AJ005061.1|
  ______________________|
 |                      |                ____ Query
 |                      |_______________|
 |                                      | gi|1257575185|dbj|LC171348.1|
 |
_, gi|1757450197|gb|MK570509.1|
 |
 | gi|1757450143|gb|MK570508.1|
 |
 | , gi|42820782|emb|AJ575642.1|
 |_|
   | gi|2784321086|gb|CP165606.1|

--------------------------------------------------
UPGMA Tree:
  _________________________________________________________ gi|3046719|emb|AJ005061.1|
 |
 |                                                                                      , gi|42820782|emb|AJ575642.1|
 |                                                                                     _|
_|                                                                                    | | gi|278

Função para ver a regulação

In [97]:
def get_regulatory_proteins(gene_id):
    """
    Fetches regulatory annotations for a gene from UniProt.
    
    Args:
        gene_id (str): UniProt ID of the gene.
    
    Returns:
        list: List of regulatory interactions.
    """
    handle = ExPASy.get_sprot_raw(gene_id)
    record = SwissProt.read(handle)
    regulatory_info = []
    for comment in record.comments:
        if "regulation" in comment.lower():
            regulatory_info.append(comment)
    if not regulatory_info:
        return "Regulatory Info not found"
    return regulatory_info

In [98]:
# Example Usage
gene_id = "P80355"  # Example UniProt ID
regulation_data = get_regulatory_proteins(gene_id)
print(regulation_data)

Regulatory Info not found


Função para identificar variantes

In [72]:
def fetch_variants_from_dbSNP(gene_id):
    """
    Fetches variant information for a gene from dbSNP via NCBI Entrez.
    
    Args:
        gene_id (str): The Gene ID of the gene of interest.
    
    Returns:
        str: Raw data from dbSNP related to the gene.
    """
    Entrez.email = "your_email@example.com"
    handle = Entrez.esearch(db="snp", term=f"{gene_id}[Gene ID]")
    record = Entrez.read(handle)
    handle.close()

    # Fetch variant data
    snp_ids = record["IdList"]
    if not snp_ids:
        return "No variants found for this gene."
    
    handle = Entrez.efetch(db="snp", id=",".join(snp_ids), rettype="xml", retmode="text")
    data = handle.read()
    handle.close()
    return data

In [74]:
gene_id = "938310" 
variants = fetch_variants_from_dbSNP(gene_id)
print(variants)

No variants found for this gene.
